In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np; np.random.seed(22)
import seaborn as sns; sns.set(color_codes=True)
import pandas as pd
import math

In [ ]:
#Import
# Imports all csv files in folder and concatonates the data sets from trackmate.

import glob

df = pd.DataFrame()
for filename in glob.glob('data2/*.csv'):
    data_01 = pd.read_csv(filename, sep=',')
    filename = filename.split('/')[1]
    filename = filename.split('.')[0]
    well = filename.split('_')[6]
    data_01['Well'] = well.split('-')[0]
    data_01['FOV'] = filename.split('_')[7]
    data = data_01[['Frame','Track_ID', 'Well', 'FOV', 'X', 'Y', 'Channel_2', 'Channel_3']]
    df = df.append(data,  ignore_index=True)
df['Sample_ID'] = df.Well + '-' + df.FOV
df['Sample_ID'] = df.Sample_ID + '-Track-' + df.Track_ID.astype(str)

In [ ]:
# Baseline Subtract
# Subtract baseline using min value: Channel_2.

def subtract_bl(in_df):
    traces = in_df
    traces_p = traces.pivot_table(index='Frame', columns='Sample_ID', values='Channel_2')
    df_test2 = pd.DataFrame()
    for columns in traces_p:
        minvalue = traces_p[columns].min()
        base_sub = lambda x: x-minvalue
        df_test = in_df[in_df['Sample_ID']==columns]
        df_test['bc_channel_1'] = df_test['Channel_2']-minvalue
        df_test2 = df_test2.append(df_test)
    return df_test2
    
df_bl = subtract_bl(df)

In [ ]:
# Baseline Subtract
# Subtract baseline using min value: Channel_3.

def subtract_bl(in_df):
    traces = in_df
    traces_p = traces.pivot_table(index='Frame', columns='Sample_ID', values='Channel_3')
    df_test2 = pd.DataFrame()
    for columns in traces_p:
        minvalue = traces_p[columns].min()
        base_sub = lambda x: x-minvalue
        df_test = in_df[in_df['Sample_ID']==columns]
        df_test['bc_channel_2'] = df_test['Channel_3']-minvalue
        df_test2 = df_test2.append(df_test)
    return df_test2
    
df_bl = subtract_bl(df_bl)

In [ ]:
# Save as Pickle
# Save baseline subtracted data to pickle file.

df_bl.to_pickle("baseLine_subtract2.pkl")

In [ ]:
# Import baseline subtracted pickle file into Pandas dataframe.

df_bl = pd.read_pickle("baseLine_subtract2.pkl")
df_f=df_bl

In [ ]:
# Filter Edges
# Filter out inclusion near the edges of the field of view.

df2 = df_f[~(df_f['X']<10)]
df2 = df2[~(df2['X']>670)]
df2 = df2[~(df2['Y']<10)]
df2 = df2[~(df2['Y']>670)]
#df2 = df_f

In [ ]:
# Calibrate Frame values from the image slices to time values of experiment.

totalFrames = 79
startTime = 12
interval = 0.5

frame_dict = {}
for i in range(totalFrames):
    if i == 0:
        frame = i 
        frame_dict[frame] = startTime+1
    else:
        frame = i 
        startTime += interval
        frame_dict[frame] = startTime+1
df2['Time'] = df2['Frame'].map(frame_dict)

In [ ]:
# Track Duration Filter
# Filter out traces that do not extend over two time points.

df_f1 = df2['Sample_ID'][df2['Time']==28]
df_f2 = df2['Sample_ID'][df2['Time']==48]
df_f3 = df_f1[df_f1.isin(df_f2)]
df3 = df2[df2['Sample_ID'].isin(df_f3)]

In [ ]:
# Assign Treatment
# Assign isolates to wells filtering for MOI < 1.

WT             = df3[df3['Sample_ID'].str.contains("B1-").fillna(False)|df3['Sample_ID'].str.contains("C1-").fillna(False)]
MUT_A2_8       = df3[df3['Sample_ID'].str.contains("B2-").fillna(False)|df3['Sample_ID'].str.contains("C2-").fillna(False)]
MUT_A3_1       = df3[df3['Sample_ID'].str.contains("B3-").fillna(False)|df3['Sample_ID'].str.contains("C3-").fillna(False)]
MUT_A3_6       = df3[df3['Sample_ID'].str.contains("B4-").fillna(False)|df3['Sample_ID'].str.contains("C4-").fillna(False)]
MUT_A3_7       = df3[df3['Sample_ID'].str.contains("B5-").fillna(False)|df3['Sample_ID'].str.contains("C5-").fillna(False)]
MUT_B1_7       = df3[df3['Sample_ID'].str.contains("B6-").fillna(False)|df3['Sample_ID'].str.contains("C6-").fillna(False)]
MUT_B1_9       = df3[df3['Sample_ID'].str.contains("B7-").fillna(False)|df3['Sample_ID'].str.contains("C7-").fillna(False)]
MUT_B1_11      = df3[df3['Sample_ID'].str.contains("B8-").fillna(False)|df3['Sample_ID'].str.contains("C8-").fillna(False)]
MUT_B2_10      = df3[df3['Sample_ID'].str.contains("B9-").fillna(False)|df3['Sample_ID'].str.contains("C9-").fillna(False)]
MUT_B2_11      = df3[df3['Sample_ID'].str.contains("A10-").fillna(False)|df3['Sample_ID'].str.contains("B10-").fillna(False)]
MUT_B2_11dot   = df3[df3['Sample_ID'].str.contains("A11-").fillna(False)|df3['Sample_ID'].str.contains("B11-").fillna(False)]
MUT_B3_6       = df3[df3['Sample_ID'].str.contains("A12-").fillna(False)|df3['Sample_ID'].str.contains("B12-").fillna(False)]

In [ ]:
# Filter for Growth
# Filter for inclusions that exhibit sufficient growth.
# Set boolean for sample ID where Max value after time of min value is > 10 frames later.

def filterII(in_df, threshold):
    traces_p =       in_df.pivot_table(index='Frame', columns='Sample_ID', values='bc_channel_2')
    df_pass = pd.DataFrame(columns=['Sample_ID', 'pass'])
    for cn in traces_p.columns:
        index_min = traces_p[cn].idxmin(axis=1, skipna=True)
        min_value = traces_p[cn].min()
        traces_p_min = traces_p[traces_p.index>=index_min]
        max_value = traces_p_min[cn].max()
        late_value = traces_p[cn].iloc[54]
        ratio = late_value/max_value
       
        if max_value > threshold*(abs(min_value)+1) and ratio > 0.01:
            df_pass = df_pass.append({'Sample_ID':cn, 'pass': True}, ignore_index=True)
            
    new_df = pd.merge(in_df, df_pass,  how='right', on=['Sample_ID'])
    return new_df
     
WT_f           =   filterII(WT            ,10000)
MUT_A2_8_f     =   filterII(MUT_A2_8      ,10000)
MUT_A3_1_f     =   filterII(MUT_A3_1      ,10000)
MUT_A3_6_f     =   filterII(MUT_A3_6      ,10000)
MUT_A3_7_f     =   filterII(MUT_A3_7      ,10000)
MUT_B1_7_f     =   filterII(MUT_B1_7      ,10000)
MUT_B1_9_f     =   filterII(MUT_B1_9      ,10000)
MUT_B1_11_f    =   filterII(MUT_B1_11     ,10000)
MUT_B2_10_f    =   filterII(MUT_B2_10     ,10000)
MUT_B2_11_f    =   filterII(MUT_B2_11     ,10000)
MUT_B2_11dot_f =   filterII(MUT_B2_11dot  ,10000)
MUT_B3_6_f     =   filterII(MUT_B3_6      ,10000)

In [ ]:
# Calculate Mean & STD
# Pivot dataframe use Time as index and Sample_ID as columns.
# Calculate mean and standard deviation of each isolate.

def pivot(in_df, channel):
    in_df_p = in_df.pivot_table(index='Time', columns='Sample_ID', values=channel)
    in_df_p['mean'], in_df_p['std'] = in_df_p.mean(axis=1), in_df_p.std(axis=1)
    return in_df_p

In [ ]:
# Pivot dataframe use Time as index and Sample_ID as columns.

EuoWT_p               = pivot(WT_f          ,   'bc_channel_1')  
EuoMUT_A2_8_p         = pivot(MUT_A2_8_f    ,   'bc_channel_1')
EuoMUT_A3_1_p         = pivot(MUT_A3_1_f    ,   'bc_channel_1')  
EuoMUT_A3_6_p         = pivot(MUT_A3_6_f    ,   'bc_channel_1')
EuoMUT_A3_7_p         = pivot(MUT_A3_7_f    ,   'bc_channel_1')  
EuoMUT_B1_7_p         = pivot(MUT_B1_7_f    ,   'bc_channel_1')
EuoMUT_B1_9_p         = pivot(MUT_B1_9_f    ,   'bc_channel_1')  
EuoMUT_B1_11_p        = pivot(MUT_B1_11_f   ,   'bc_channel_1')
EuoMUT_B2_10_p        = pivot(MUT_B2_10_f   ,   'bc_channel_1')  
EuoMUT_B2_11_p        = pivot(MUT_B2_11_f   ,   'bc_channel_1')
EuoMUT_B2_11dot_p     = pivot(MUT_B2_11dot_f,   'bc_channel_1')  
EuoMUT_B3_6_p         = pivot(MUT_B3_6_f    ,   'bc_channel_1')


HctBWT_p              = pivot(WT_f          ,   'bc_channel_2')  
HctBMUT_A2_8_p        = pivot(MUT_A2_8_f    ,   'bc_channel_2')
HctBMUT_A3_1_p        = pivot(MUT_A3_1_f    ,   'bc_channel_2')  
HctBMUT_A3_6_p        = pivot(MUT_A3_6_f    ,   'bc_channel_2')
HctBMUT_A3_7_p        = pivot(MUT_A3_7_f    ,   'bc_channel_2')  
HctBMUT_B1_7_p        = pivot(MUT_B1_7_f    ,   'bc_channel_2')
HctBMUT_B1_9_p        = pivot(MUT_B1_9_f    ,   'bc_channel_2')  
HctBMUT_B1_11_p       = pivot(MUT_B1_11_f   ,   'bc_channel_2')
HctBMUT_B2_10_p       = pivot(MUT_B2_10_f   ,   'bc_channel_2')  
HctBMUT_B2_11_p       = pivot(MUT_B2_11_f   ,   'bc_channel_2')
HctBMUT_B2_11dot_p    = pivot(MUT_B2_11dot_f,   'bc_channel_2')  
HctBMUT_B3_6_p        = pivot(MUT_B3_6_f    ,   'bc_channel_2')

In [ ]:
#Check filtering threshold.

EuoMUT_B1_9_p.plot(legend=False)
EuoWT_p.plot(legend=False)

In [ ]:
# Select color palette for graph.

c = sns.color_palette('Set1',16).as_hex()
c[1]

In [ ]:
# Graph Iso vs WT
# Graph each isolate against wt strain using mean and SEM.

with plt.style.context('seaborn-white'):
    fig, ((ax2,ax3),(ax4,ax5),(ax6,ax7),(ax8,ax9),(ax10,ax11),(ax12,ax13)) = plt.subplots(ncols=2, nrows=6)

    def plot_sample_2(sample, color, name, style, mstyle, fcolor, i):
        ax2.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)     
        ax2.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)
    
    plot_sample_2(EuoWT_p , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_2(HctBWT_p, c[1], 'HctB WT' , '--', '', 'None', 1)
       
    def plot_sample_3(sample, color, name, style, mstyle, fcolor, i):
        ax3.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax3.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
  
    plot_sample_3(EuoWT_p       , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_3(HctBWT_p      , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_3(EuoMUT_A3_1_p , c[2], 'Euo MUT A3_1' , '-', '', 'None', 1)
    plot_sample_3(HctBMUT_A3_1_p, c[3], 'HctB MUT A3_1' , '-', '', 'None', 1)

    
    def plot_sample_4(sample, color, name, style, mstyle, fcolor, i):
        ax4.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax4.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
    
    plot_sample_4(EuoWT_p       , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_4(HctBWT_p      , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_4(EuoMUT_A3_7_p , c[2], 'EuoMUT A3_7' , '-', '', 'None', 1)
    plot_sample_4(HctBMUT_A3_7_p, c[3], 'HctBMUT A3_7' , '-', '', 'None', 1)
    
    def plot_sample_5(sample, color, name, style, mstyle, fcolor, i):
        ax5.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax5.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
    
    plot_sample_5(EuoWT_p       , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_5(HctBWT_p      , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_5(EuoMUT_A3_6_p , c[2], 'Euo MUT_A3_6' , '-', '', 'None', 1)
    plot_sample_5(HctBMUT_A3_6_p, c[3], 'HctB MUT_A3_6' , '-', '', 'None', 1)
    
    def plot_sample_6(sample, color, name, style, mstyle, fcolor, i):
        ax6.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax6.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
    
    plot_sample_6(EuoWT_p       , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_6(HctBWT_p      , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_6(EuoMUT_B1_7_p , c[2], 'Euo MUT B1_7' , '-', '', 'None', 1)
    plot_sample_6(HctBMUT_B1_7_p, c[3], 'HctB MUT B1_7' , '-', '', 'None', 1)
    
    def plot_sample_7(sample, color, name, style, mstyle, fcolor, i):
        ax7.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax7.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
    
    plot_sample_7(EuoWT_p       , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_7(HctBWT_p      , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_7(EuoMUT_B1_9_p , c[2], 'Euo MUT B1_9' , '-', '', 'None', 1)
    plot_sample_7(HctBMUT_B1_9_p, c[3], 'HctB MUT B1_9' , '-', '', 'None', 1)

    def plot_sample_8(sample, color, name, style, mstyle, fcolor, i):
        ax8.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax8.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
    
    plot_sample_8(EuoWT_p       , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_8(HctBWT_p      , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_8(EuoMUT_A2_8_p , c[2], 'EuoMUT A2_5' , '-', '', 'None', 1)
    plot_sample_8(HctBMUT_A2_8_p, c[3], 'HctBMUT A2_5' , '-', '', 'None', 1)
    
    def plot_sample_9(sample, color, name, style, mstyle, fcolor, i):
        ax9.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax9.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
    
    plot_sample_9(EuoWT_p        , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_9(HctBWT_p       , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_9(EuoMUT_B1_11_p , c[2], 'Euo MUT B1_11' , '-', '', 'None', 1)
    plot_sample_9(HctBMUT_B1_11_p, c[3], 'HctB MUT B1_11' , '-', '', 'None', 1)
    
    def plot_sample_10(sample, color, name, style, mstyle, fcolor, i):
        ax10.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax10.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
    
    plot_sample_10(EuoWT_p        , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_10(HctBWT_p       , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_10(EuoMUT_B2_10_p , c[2], 'Euo MUT B2_10' , '-', '', 'None', 1)
    plot_sample_10(HctBMUT_B2_10_p, c[3], 'HctB MUT B2_10' , '-', '', 'None', 1)
    
    def plot_sample_11(sample, color, name, style, mstyle, fcolor, i):
        ax11.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax11.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
    
    plot_sample_11(EuoWT_p        , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_11(HctBWT_p       , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_11(EuoMUT_B2_11_p , c[2], 'Euo MUT B2_11' , '-', '', 'None', 1)
    plot_sample_11(HctBMUT_B2_11_p, c[3], 'HctB MUT B2_11' , '-', '', 'None', 1)
    
    def plot_sample_12(sample, color, name, style, mstyle, fcolor, i):
        ax12.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax12.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
    
    plot_sample_12(EuoWT_p           , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_12(HctBWT_p          , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_12(EuoMUT_B2_11dot_p , c[2], 'EuoMUT B2_11dot' , '-', '', 'None', 1)
    plot_sample_12(HctBMUT_B2_11dot_p, c[3], 'HctBMUT B2_11dot' , '-', '', 'None', 1)
    
    def plot_sample_13(sample, color, name, style, mstyle, fcolor, i):
        ax13.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)
        ax13.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)    
    
    plot_sample_13(EuoWT_p       , c[0], 'Euo WT' , '--', '', 'None', 1)
    plot_sample_13(HctBWT_p      , c[1], 'HctB WT' , '--', '', 'None', 1)
    plot_sample_13(EuoMUT_B3_6_p , c[2], 'Euo MUT B3_6' , '-', '', 'None', 1)
    plot_sample_13(HctBMUT_B3_6_p, c[3], 'HctB MUT B3_6' , '-', '', 'None', 1)

with plt.style.context('classic'):
    ax2.legend(loc='upper left', fontsize=4)
    ax3.legend(loc='upper left', fontsize=4)
    ax4.legend(loc='upper left', fontsize=4)
    ax5.legend(loc='upper left', fontsize=4)
    ax6.legend(loc='upper left', fontsize=4)
    ax7.legend(loc='upper left', fontsize=4)
    ax8.legend(loc='upper left', fontsize=4)
    ax9.legend(loc='upper left', fontsize=4)
    ax10.legend(loc='upper left', fontsize=4)
    ax11.legend(loc='upper left', fontsize=4)
    ax12.legend(loc='upper left', fontsize=4)
    ax13.legend(loc='upper left', fontsize=4)
    ax2.set_ylabel('Relative Fluorescence')
    ax2.set_xlabel('HPI')
    ax3.set_xlabel('HPI')
    ax4.set_xlabel('HPI')
    ax5.set_xlabel('HPI')
    ax6.set_xlabel('HPI')
    ax7.set_xlabel('HPI')
    ax8.set_xlabel('HPI')
    ax9.set_xlabel('HPI')
    ax10.set_xlabel('HPI')
    ax11.set_xlabel('HPI')
    ax12.set_xlabel('HPI')
    ax13.set_xlabel('HPI')
    
    #ax2.set_xlim([12, 48])
    #ax3.set_xlim([12, 48])
    #ax4.set_xlim([12, 48])

   
    fig.set_size_inches(6, 24)
    #plt.savefig('MutantVerification.pdf')

In [ ]:
# Calculate
# Calculate max, min, halfmax and time to halfmax.

def halfmax(in_pd):
    in_pd_c1_p = in_pd.pivot_table(index='Sample_ID', columns='Time', values='bc_channel_1')
    in_pd_c1_p['max_c1'] = in_pd_c1_p.max(axis=1)
    in_pd_c2_p = in_pd.pivot_table(index='Sample_ID', columns='Time', values='bc_channel_2')
    in_pd_c2_p['max_c2'] = in_pd_c2_p.max(axis=1)

    in_pd_c1_p['min_c1'] = 1
    in_pd_c2_p['min_c2'] = 1
    in_pd_c1_p['half_max_c1'] = in_pd_c1_p['max_c1']/2
    in_pd_c2_p['half_max_c2'] = in_pd_c2_p['max_c2']/2
  
    in_pd_f = in_pd_c1_p[['min_c1','max_c1', 'half_max_c1']]
    in_pd_f['min_c2'] = in_pd_c2_p['min_c2']
    in_pd_f['max_c2'] = in_pd_c2_p['max_c2']
    in_pd_f['half_max_c2'] = in_pd_c2_p['half_max_c2']
    in_pd_f.index.name = None
    in_pd_f['Sample_ID'] = in_pd_f.index

    traces = in_pd
    
    traces_c1_p =       traces.pivot_table(index='Time', columns='Sample_ID', values='bc_channel_1') 
    traces_c1_p_fill =  traces_c1_p.fillna(method='bfill') #back fill to frame 0
    traces_c1_p_fill.reset_index(inplace = True)
    df_c1_f = traces_c1_p_fill
    df_c1_f.set_index(df_c1_f['Time'], inplace=True)
    
    traces_c2_p =       traces.pivot_table(index='Time', columns='Sample_ID', values='bc_channel_2') 
    traces_c2_p_fill =  traces_c2_p.fillna(method='bfill') #back fill to frame 0
    traces_c2_p_fill.reset_index(inplace = True)
    df_c2_f = traces_c2_p_fill
    df_c2_f.set_index(df_c2_f['Time'], inplace=True)
    
    time_halfmax_c1 = pd.DataFrame(columns=['Sample_ID', 'time_to_halfmax_c1'])
    time_halfmax_c2 = pd.DataFrame(columns=['Sample_ID', 'time_to_halfmax_c2'])
    for index, row in in_pd_f.iterrows():
        sampleID = row['Sample_ID']
      
        time_c1 = (df_c1_f[sampleID]-row['half_max_c1']).abs().sort_values().index[0]
        time_c2 = (df_c2_f[sampleID]-row['half_max_c2']).abs().sort_values().index[0]
      
        time_halfmax_c1 = time_halfmax_c1.append({'Sample_ID':sampleID, 'time_to_halfmax_c1': time_c1}, ignore_index=True)
        time_halfmax_c2 = time_halfmax_c2.append({'Sample_ID':sampleID, 'time_to_halfmax_c2': time_c2}, ignore_index=True)
       
    data_summary = pd.merge(time_halfmax_c1, time_halfmax_c2, on='Sample_ID')
    # Add x y coordinates at last time imaged.
    df_time = df3[df3['Time']==35]
    df_time = df_time[['Sample_ID', 'X', 'Y']]
    # Data_summary only for Sample IDs in both
    result = pd.merge(data_summary, df_time, how='inner', on=['Sample_ID'])
    return result


WT_hm             = halfmax(WT_f          )

MUT_A2_8_hm       = halfmax(MUT_A2_8_f    )
MUT_A3_1_hm       = halfmax(MUT_A3_1_f    )
MUT_A3_6_hm       = halfmax(MUT_A3_6_f    )
MUT_A3_7_hm       = halfmax(MUT_A3_7_f    )
MUT_B1_7_hm       = halfmax(MUT_B1_7_f    )
MUT_B1_9_hm       = halfmax(MUT_B1_9_f    )
MUT_B1_11_hm      = halfmax(MUT_B1_11_f   )
MUT_B2_10_hm      = halfmax(MUT_B2_10_f   )
MUT_B2_11_hm      = halfmax(MUT_B2_11_f   )
MUT_B2_11dot_hm   = halfmax(MUT_B2_11dot_f)
MUT_B3_6_hm       = halfmax(MUT_B3_6_f    )

In [ ]:
# Visualize idividual traces in interactive graph.

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook, output_file
from bokeh.layouts import row
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
from bokeh.models import HoverTool
from collections import OrderedDict
from bokeh.layouts import layout
from bokeh.models import Toggle, BoxAnnotation, CustomJS

output_notebook()

c = sns.color_palette('Set1',1000).as_hex()

def interactive_graph(in_df):
    in_df_p = pivot(in_df, 'bc_channel_2')
    df_new = in_df_p[in_df_p.columns[0:-2]]
    result = halfmax(in_df)
    
    tools_to_show = 'box_zoom,save,hover,reset'
    p = figure(plot_height = 500, plot_width = 500,
               toolbar_location='above', tools=tools_to_show,
               # "easy" tooltips in Bokeh 0.13.0 or newer
               tooltips=[("Location","$name")])
    j=0
    for i, column in enumerate(df_new):
        trackx = result[result['Sample_ID'] == df_new.iloc[:,i].name].X.values
        tracky = result[result['Sample_ID'] == df_new.iloc[:,i].name].Y.values
        x = str(int(trackx))
        y = str(int(tracky))
        well = df_new.iloc[:,i].name.split('-')[0]
        field = df_new.iloc[:,i].name.split('-')[1]
        well = df_new.iloc[:,i].name
        name = well+', x='+x+', y='+y
        p.line(df_new.index.values, df_new.iloc[:,i].values, name = name, line_color=c[j], line_width=2.5)
        j=j+1
    p.legend.location = "top_left"
    p.legend.click_policy="mute"
    show(p)
interactive_graph(MUT_B3_6_f )

In [ ]:
# Half-Max Plot
# Visualize time to half-maximal expression of each promoter for individual inclusions.
# Identify location of inclusion outliers.

def spot_plot(title, in_df, in_df2):
    TOOLTIPS = [("ID", "@Sample_ID"), ('X', '@X'), ('Y', '@Y')]
    p = figure(title=title, tooltips=TOOLTIPS, y_range=(15, 60), x_range=(15,60), plot_width=600, plot_height=300)
    source = ColumnDataSource(in_df)
    source2= ColumnDataSource(in_df2)
    p.circle(x='time_to_halfmax_c1', y='time_to_halfmax_c2', size=10, color='green', alpha=0.25, source=source)
    p.circle(x='time_to_halfmax_c1', y='time_to_halfmax_c2', size=10, color='blue', alpha=0.25, source=source2)
    show(p)
    
spot_plot('EMS', MUT_B2_11_hm, WT_hm)

In [ ]:
# Animated Plot
# Visualize promoter expression kientics of each inclusion through time.

WT_f['treatment']='none'
MUT_B2_11_f['treatment']='EMS'
df3 = pd.concat([WT_f, MUT_B2_11_f])
df3.sort_values('Time', inplace=True)

import plotly.express as px
gapminder = px.data.gapminder()
fig = px.scatter(df3, x="bc_channel_1", y="bc_channel_2", animation_frame="Time",
           hover_name="Sample_ID", animation_group="Sample_ID", color='treatment',
           log_x=False, size_max=85, range_x=[0, 60000], range_y=[-500, 20000], color_discrete_map = {"none": 'rgba(0, 0, 255, .3)', "EMS": 'rgba(0, 255, 0, .3)'})
fig.update_traces(marker=dict(size=6))
fig.show()

In [ ]:
# Inclusion Locator
# Visualize euo vs hctB expression at 32 HPI for individual inclusions.
# Identify location of inclusion outliers.

MT32 = MUT_B3_6_f[MUT_B3_6_f['Time']==32] # Get values at specified HPI for all sample IDs.
WT32 = WT_f[WT_f['Time']==32] # Get values at specified HPI for all sample IDs.
def spot_plot(title, in_df, in_df2):
    TOOLTIPS = [("ID", "@Sample_ID"), ('X', '@X'), ('Y', '@Y')]
    p = figure(title=title, tooltips=TOOLTIPS, x_range=(0, 60000), y_range=(-1000, 12000), plot_width=600, plot_height=300)
    source = ColumnDataSource(in_df)
    source2= ColumnDataSource(in_df2)
    p.circle(x='bc_channel_1', y='bc_channel_2', size=10, color='green', alpha=0.25, source=source)
    p.circle(x='bc_channel_1', y='bc_channel_2', size=10, color='blue', alpha=0.25, source=source2)
    show(p)
    
spot_plot('EUO vs HctB @32 HPI', MT32, WT32)